In [ ]:
import os
from glob import glob
from PIL import Image
import numpy as np
import json
import random
from copy import deepcopy
from tqdm import tqdm
import cv2

In [ ]:
# YOLO 형식 txt 파일 생성 함수
def crop_label(crop_h,crop_w,label,input_size):
    # Crop image
    #args: crop_h, crop_w, image, label, input_size
    temp_labels = []
    for k in range(len(label['annotations'])):
        x = label['annotations'][k]['bbox'][0]
        y = label['annotations'][k]['bbox'][1]
        w = label['annotations'][k]['bbox'][2]
        h = label['annotations'][k]['bbox'][3]
        if x >= crop_h+4 and y >= crop_w+4 and x <= crop_h + input_size-4 and y <= crop_w + input_size-4:
            label['annotations'][k]['bbox'][0] = (x - crop_h) 
            label['annotations'][k]['bbox'][1] = (y - crop_w) 
            temp_labels.append(label['annotations'][k])
    return temp_labels

def create_yolo_txt(crop_labels, filename, class_name, input_size=512):
    """
    YOLO 형식의 라벨 txt 파일 생성
    Args:
        crop_labels: 크롭된 라벨 리스트
        filename: 이미지 파일명 (확장자 제외)
        class_name: 'train' or 'val'
    """
    txt_path = f'../../data/yolo_IGNITE/labels/{class_name}/{filename}.txt'
    
    yolo_lines = []
    for label in crop_labels:
        # COCO bbox: [x, y, width, height] (절대좌표)
        # YOLO bbox: [x_center, y_center, width, height] (정규화된 0~1 사이 값)
        
        x, y, w, h = label['bbox']
        
        # 중심점 계산 (이미 정규화된 상태)
        x_center = x + w/2
        y_center = y + h/2
        
        # 클래스 ID (COCO는 1부터 시작, YOLO는 0부터 시작)
        class_id = label['category_id'] - 1
        
        # YOLO 형식: class_id x_center y_center width height
        yolo_line = f"{class_id} {max(0,min(1,x_center/input_size)):.6f} {max(0,min(1,y_center/input_size)):.6f} {max(0,min(1,w/input_size)):.6f} {max(0,min(1,h/input_size)):.6f}"
        yolo_lines.append(yolo_line)
    
    # txt 파일 저장
    with open(txt_path, 'w') as f:
        f.write('\n'.join(yolo_lines))

def custom_yolo_creation(json_data, file_path, input_size=512, class_name='train'):
    """
    YOLO 데이터셋 생성 (이미지 + txt 라벨)
    """
    w = json_data['image']['width']
    h = json_data['image']['height']
    r = input_size / min(h, w)
    image = Image.open(os.path.join(file_path, json_data['image']['file_name']))
    
    base_filename = json_data['image']['file_name'].split('.')[0]
    
    if r < 1:
        # 이미지가 input_size보다 큰 경우 → 크롭
        h_count = h // input_size
        w_count = w // input_size
        
        for hi in range(h_count):
            for wi in range(w_count):
                h1 = hi * input_size
                w1 = wi * input_size
                
                # 경계 조정
                if h1 + input_size > h:
                    h1 = h - input_size
                if w1 + input_size > w:
                    w1 = w - input_size
                
                # 라벨 크롭
                crop_labels = crop_label(h1, w1, json_data, input_size)
                
                if len(crop_labels) > 0:
                    # 이미지 크롭 및 저장
                    crop_image = image.crop((w1, h1, w1 + input_size, h1 + input_size))
                    img_filename = f'{base_filename}_{hi}_{wi}'
                    crop_image.save(f'../../data/yolo_IGNITE/images/{class_name}/{img_filename}.png')
                    
                    # YOLO txt 라벨 생성
                    create_yolo_txt(crop_labels, img_filename, class_name)
                    
    
    else:
        # 이미지가 input_size보다 작거나 같은 경우 → 패딩
        h1, w1 = 0, 0
        crop_labels = crop_label(h1, w1, json_data, input_size)
        
        if len(crop_labels) > 0:
            # 패딩된 이미지 생성
            pad_image = np.ones((input_size, input_size, 3), dtype=np.uint8) * 255
            pad_image[:min(h, input_size), :min(w, input_size), :] = np.array(image)[:min(h, input_size), :min(w, input_size), :3]
            
            # 이미지 저장
            img_filename = f'{base_filename}_{h1}_{w1}'
            cv2.imwrite(f'../../data/yolo_IGNITE/images/{class_name}/{img_filename}.png', pad_image)
            
            # YOLO txt 라벨 생성
            create_yolo_txt(crop_labels, img_filename, class_name)
            

print("YOLO 형식 생성 함수 준비 완료!")

In [ ]:
# YOLO 데이터셋 생성 실행
json_list = glob('../../data/IGNITE/annotations/pdl1/individual/*.json')
file_path = '../../data/IGNITE/images/pdl1/pdl1/'

# 폴더 생성
os.makedirs('../../data/yolo_IGNITE/images/train/', exist_ok=True)
os.makedirs('../../data/yolo_IGNITE/images/val/', exist_ok=True)
os.makedirs('../../data/yolo_IGNITE/labels/train/', exist_ok=True)
os.makedirs('../../data/yolo_IGNITE/labels/val/', exist_ok=True)


# 데이터셋 생성
train_count = 0
val_count = 0

for i in tqdm(range(len(json_list)), desc="YOLO 데이터셋 생성 중"):
    # 80% train, 20% val 분할
    dataset_classification = random.randint(0, 9)
    
    with open(json_list[i], 'r') as f:
        data = json.load(f)
    
    try:
        if dataset_classification < 8:  # 80% → train
            class_name = 'train'
            custom_yolo_creation(data, file_path, input_size=512, class_name=class_name)
            train_count += 1
        else:  # 20% → val
            class_name = 'val'
            custom_yolo_creation(data, file_path, input_size=512, class_name=class_name)
            val_count += 1
    except Exception as e:
        print(f"❌ 오류 발생 ({json_list[i]}): {e}")

print(f"\n🎯 YOLO 데이터셋 생성 완료!")
print(f"📈 Train: {train_count}개 원본 이미지")
print(f"📊 Val: {val_count}개 원본 이미지")

In [ ]:
# YOLO 데이터셋 검증
def validate_yolo_dataset():
    """생성된 YOLO 데이터셋 검증"""
    
    for split in ['train', 'val']:
        img_dir = f'../../data/yolo_IGNITE/images/{split}/'
        label_dir = f'../../data/yolo_IGNITE/labels/{split}/'
        
        # 이미지와 라벨 파일 수 확인
        img_files = glob(os.path.join(img_dir, '*.png'))
        txt_files = glob(os.path.join(label_dir, '*.txt'))
        
        print(f"\n📊 {split.upper()} 데이터셋:")
        print(f"  - 이미지: {len(img_files)}개")
        print(f"  - 라벨: {len(txt_files)}개")
        
        # 라벨 통계
        total_objects = 0
        class_counts = [0, 0, 0]  # 3개 클래스
        
        for txt_file in txt_files:
            with open(txt_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.strip():
                        class_id = int(line.split()[0])
                        class_counts[class_id] += 1
                        total_objects += 1
        
        print(f"  - 총 객체 수: {total_objects}개")
        print(f"  - Class 0 (pd-l1 negative): {class_counts[0]}개")
        print(f"  - Class 1 (pd-l1 positive): {class_counts[1]}개") 
        print(f"  - Class 2 (non-tumor): {class_counts[2]}개")

validate_yolo_dataset()

# YOLO YAML 설정 파일 생성
yaml_content = """# YOLO 데이터셋 설정
path: /home/work/IHC_biomarker/data/yolo_IGNITE
train: images/train
val: images/val
test: images/val

# 클래스 수
nc: 3

# 클래스 이름 (0부터 시작)
names:
  0: pd-l1 negative tumor cell
  1: pd-l1 positive tumor cell
  2: non-tumor cell
"""

# YAML 파일 저장
with open('IGNITE_yolo.yaml', 'w') as f:
    f.write(yaml_content)

print(f"\n✅ YOLO 설정 파일 생성: IGNITE_yolo.yaml")
print(f"🚀 이제 다음 명령으로 학습할 수 있습니다:")
print(f"   from ultralytics import RTDETR")
print(f"   model = RTDETR('rtdetr-l.pt')")
print(f"   model.train(data='IGNITE_yolo.yaml', epochs=100)")